In [1]:
from zeep import Client
import zeep
from zeep.wsse.username import UsernameToken

In [ ]:
client = Client('https://servicoshm.saude.gov.br/cnes/EstabelecimentoSaudeService/v1r0?wsdl', wsse=UsernameToken('CNES.PUBLICO', 'cnes#2015public'))

In [50]:
request_data = {
    'FiltroPesquisaEstabelecimentoSaude': {
        'CodigoCNES': {
            'codigo': 2653982
        }
    },
}

In [6]:
import pandas
import time

In [7]:
data_frame = pandas.read_csv('listas/municipios.csv', sep=';')

In [8]:
client_cnes_service = Client('https://servicoshm.saude.gov.br/cnes/CnesService/v1r0?wsdl', wsse=UsernameToken('CNES.PUBLICO', 'cnes#2015public'))

In [ ]:
for row in data_frame.itertuples():
    file_name = 'resultados/%s/%s/estabelecimentos.csv' % (row.UF, row.MUNICIPIO) 
    
    request_data_municipio = {
        'Municipio': {
            'codigoMunicipio': row.MUNICIPIO
        }
    }
    
    response = client_cnes_service.service.consultarEstabelecimentoSaudePorMunicipio(**request_data_municipio)
    serialized_response = zeep.helpers.serialize_object(response[0])
    
    response_df = pandas.DataFrame(serialized_response['DadosBasicosEstabelecimento'])
    response_df['codigo_tratado'] = response_df.CodigoCNES.apply(lambda x: x['codigo'])
    response_df['nome_tratado'] = response_df.NomeFantasia.apply(lambda x: x['Nome'])
    
    response_df.to_csv(file_name, encoding='utf-8', index=False, columns=['codigo_tratado','nome_tratado'])
    
    time.sleep(5)
    